## Dobble
A simple implementation of the Dobble card game has is created. Each card has eight images and each pair of cards will have exactly one image in common. The game is to be the first to spot the matching image on each pair. For the game, one card is revealed each time and compared with the last card in the game. This way, each card is involved twice. At the beginning of the game, players decide how many rounds they want to play in the game. Players can record who won with 'A' or 'B', and a draw can be recorded with a 'D' or 'd'.

### Files
An emoji package is used for this game. To install please use: pip install emoji --upgrade
This game uses a .text file (emoji_names) with 57 emoji's. The original required file had 60, but 3 names did not presented an emoji from the package and therefor they have been removed.

### Structure

**image_dictionary:** 
57 emoji's are stored in image_dictionary

**card_dictionary:** 
Generates cards with sets of image_ids and stores them in dictionary with the card number as key and the image id's (in a set) as the value.

**Class DoubleCard*:** 
A simple set blueprint for the cards. There are 57 valid cards.

**Class DobbleDeck*:** 
It holds and manages a deck of cards that are stored as DoubleCard instances ad has three methods:

    1. add_card*
    2. remove_card*
    3. play_card*

**Function generate_cards:** 
Generates cards with sets of image_ids and stores them in card_dictionary. 
For the generation of the pairs with only one matching image, some sample code has been given and edited. It works only when the number of images on the card is a primenumber plus 1, e.g. 3,4,6,8. The program is designed to be as flexible as possible for future changes and updates. Upon delivery, the function 'generate_cards' takes 8 images as requested, but this can be quickly adjusted with an argument when calling the function. The amount of available cards can vary for each image number which should be taken into account when adjusting the game.

**Function check_validity:**
A check_validity function is created that will take the deck as argument and check if the cards only match on one image. The intersection method is used here for the validation. This function is callable in two modes:
1. check_validity(deck, verbose = True)
2. check_validity(deck)
The first one produces output after validation.

**Function print_card_format:** 
Casts the image_ids to an emoji with the image_dictionary

**Function print_cards:**
Calls the print_card_format

**Function dobble_game*:**
Allows two users to play. It will present 'cards' as lists of emoji's by calling the format function. The users will decide who wins and provide the input to record the win counts.


*This class, method or function have been created by requirement 


In [13]:
# Import modules
import random
import emoji

# Create image dictionary from file that will be loaded that contains the specific emoji names (57x) that we want
image_dictionary = dict()

filename = open('emoji_names.txt',"r")
lines = filename.readlines()

# Looping through the lines in the file and make additions to the dictionary
for i, el in enumerate(lines):
    image_dictionary[i+1] = emoji.emojize(el.strip())

In [14]:
# We create blueprint for the DobbleCards
class DobbleCard:

    def __init__(self, total_images, card_id, set_of_image_ids):
        self.total_images = total_images
        self.card_id = card_id
        self.set_of_image_ids = set_of_image_ids

In [15]:
# We create blueprint and methods for creating and managing the Deck (adding, removing, playing cards)
class DobbleDeck:
    
    # Deck blueprint
    def __init__(self):
        self.deck = []
        self.__in_game = []
        pass
    
    # Method to add card to the deck
    def add_card(self, card):
        self.deck.append(card)

    # Method to remove a card from the deck
    def remove_card(self, card_id):
        card_index = -1
        for index, card in enumerate(self.deck):
            if card.card_id == card_id:
                card_index = index

        if self.deck[card_index].card_id in self.__in_game:
            self.__in_game.remove(self.deck[card_index].card_id)
        del self.deck[card_index]

    # Method to play a random card that is still in the game
    def play_card(self):
        random.shuffle(self.deck)

        if len(self.__in_game) == 0:
            to_play = self.deck[0]
            self.__in_game.append(to_play.card_id)
            return self.deck[0]
        else:
            playable = list(filter(lambda x: x.card_id not in self.__in_game, self.deck))
            if len(playable) > 0:
                self.__in_game.append(playable[0].card_id)
                return playable[0]
            else:
                return None

In [16]:
# With this function we generate cards and make them accessable in card_dictionary
def generate_cards(total_images):
    card_dictionary = {}
    n = total_images - 1
    r = range(n)
    r_plus1 = range(n + 1)
    card_index = 1

    set_of_image_ids = [i + 1 for i in r_plus1]
    card_dictionary[card_index] = set(set_of_image_ids)

    for j in r:
        card_index += 1

        set_of_image_ids = [n + 2 + n * j + k for k in r]
        set_of_image_ids.append(1)
        card_dictionary[card_index] = set(set_of_image_ids)

    for i in r:
        for j in r:
            card_index += 1

            set_of_image_ids = [(n + 1 + n * k + (i * k + j) % n) + 1 for k in r]
            set_of_image_ids.append(i + 2)
            card_dictionary[card_index] = set(set_of_image_ids)

    return card_dictionary

In [17]:
# Check if cards are indeed valid
def check_validity(deck, verbose=True):
    card_ids = list(deck.keys())
    is_valid = True

    for index, card_id in enumerate(card_ids):
        remaining_ids = card_ids[index + 1:]
        current_set_of_image_ids = deck[card_id]

        for e in remaining_ids:
            s = deck[e]
            if verbose:
                print('** Comparing card {} with card {}:'.format(card_id, e), end=' ')
            if len(current_set_of_image_ids.intersection(s)) != 1:
                print('Mismatch!')
                is_valid = False
            else:
                if verbose:
                    print('Valid!')

    return is_valid

In [18]:
# Printing the format in which we will print the cards later
def print_card_format(format_card1, format_card2):
    card1 = list(format_card1.set_of_image_ids)
    card2 = list(format_card2.set_of_image_ids)

    # Print card 1 and card 2 next to each other with 3 images in a row by default 
    for i in range(0,len(card1)-3,3):
        print(image_dictionary[card1[i]], image_dictionary[card1[i+1]], image_dictionary[card1[i+2]], '\t', image_dictionary[card2[i]], image_dictionary[card2[i+1]], image_dictionary[card2[i+2]])
    
    remainder = len(card1) % 3
    if remainder == 2:
        print(image_dictionary[card1[len(card1)-2]], image_dictionary[card1[len(card1)-1]], '\t\t', image_dictionary[card2[len(card2)-2]], image_dictionary[card2[len(card2)-1]])
    elif remainder == 1:
        print(image_dictionary[card1[len(card1)-1]], '\t\t', image_dictionary[card2[len(card2)-1]])

# Printing the content of the cards        
def print_cards(card1, card2):
    content_card1 = set()
    content_card2 = set()
    
    # Cast sets of cards to a list and add the matching images from the image_dictionary
    for value in list(card1.set_of_image_ids):
        content_card1.add(image_dictionary[value])
    for value in list(card2.set_of_image_ids):
        content_card2.add(image_dictionary[value])
    
    print_card_format(card1, card2)        

In [19]:
# One function that plays the game calling the other functions and methods
def dobble_game(deck, rounds):
  
    print('If you want to record a draw type \'d\', \'D\' or something else. \n')
    score_a = 0
    score_b = 0

    card_1 = None
    card_2 = None

    for round in range(int(rounds)):
        if card_1 is None:
            card_1 = deck.play_card()
            card_2 = deck.play_card()
        else:
            deck.remove_card(card_1.card_id)
            card_1 = card_2
            card_2 = deck.play_card()

        print_cards(card_1, card_2)

        result = input('Who wins (A or B)? ').lower()
        print()
        
        if result == 'a':
            score_a += 1
        elif result == 'b':
            score_b += 1
        
    print('Score\nA: {}\nB: {}'.format(score_a, score_b))

In [20]:
# Call the generate-cards function with 8 images for each card (as per requirement). 
cards = generate_cards(8)

# Check validity of the cards and evt print error message
is_valid = check_validity(cards, verbose=False)

if not is_valid:
    print('Deck is not valid!')

# Call DobbleDeck function to create a deck with all it's methods
deck = DobbleDeck()

for card in cards.items():
    card_id = card[0]
    card_images = card[1]
    card_object = DobbleCard(len(card_images), card_id, card_images)
    deck.add_card(card_object)

# Ask user how many card rounds they want to play and run the game with that amount of rounds
rounds = input('How many rounds do you want to play (<56)? ')
dobble_game(deck, rounds)

How many rounds do you want to play (<56)? 4
If you want to record a draw type 'd', 'D' or something else. 

🐕 🍎 ⛄ 	 🍎 🚑 🌟
🐜 🚢 🎩 	 🐴 🚽 🎺
✊ 🍉 		 💧 🍓
Who wins (A or B)? f

🍎 🚑 🌟 	 🌴 🚓 🐴
🐴 🚽 🎺 	 ⛺ ⚓ 🎸
💧 🍓 		 💔 🍉
Who wins (A or B)? f

🌴 🚓 🐴 	 🌵 🚓 🐰
⛺ ⚓ 🎸 	 ⛄ ⏰ 🎵
💔 🍉 		 🎈 💧
Who wins (A or B)? f

🌵 🚓 🐰 	 🍌 🐸 🐢
⛄ ⏰ 🎵 	 🚢 🎸 💧
🎈 💧 		 🍺 🍀
Who wins (A or B)? f

Score
A: 0
B: 0
